In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pip install -q -r requirements.txt


[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import json
import pandas as pd
import os
import re
import string
import pickle

from transformers import BertForSequenceClassification
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import AutoTokenizer
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
import os
#os.chdir('../..')

from vecsim_app.categories import CATEGORIES
from vecsim_app.data_utils import papers

Define the parameters for fetching the papers dataset:

- Dataset Path
- Year cutoff: Year cut off for the papers.
- Pattern for fetching a given amount of years
- Max Sample size: maximum simple size (if you just want to try out the notebook - if it's too low the model won't perform well)

In [8]:
DATA_PATH = "/home/paulo/redis_pr/redis-arXiv-search/data/multilabel_classifier/arxiv-metadata-oai-snapshot.json"
YEAR_CUTOFF = 2010
YEAR_PATTERN = r"(19|20[0-9]{2})"
MAX_SAMPLE_SIZE = 5000


In [9]:
df = pd.DataFrame(papers(data_path=DATA_PATH, year_cutoff=YEAR_CUTOFF, year_pattern=YEAR_PATTERN))
len(df)

# Take a sample for computing reasons
df = df.sample(MAX_SAMPLE_SIZE)

In [15]:
df.head(3)

,id,title,year,authors,categories,abstract
0,0704.0199,Decomposition numbers for finite Coxeter group...,2010,"Christian Krattenthaler (Universit\""at Wien) a...","math.CO,math.GR",Given a finite irreducible Coxeter group $W$...
1,0704.0304,The World as Evolving Information,2012,Carlos Gershenson,"cs.IT,cs.AI,math.IT,q-bio.PE",This paper discusses the benefits of describ...
2,0704.0367,Instanton representation of Plebanski gravity....,2011,Eyo Eyo Ita III,gr-qc,The instanton representation of Plebanski gr...


In [11]:
df['text'] = df['title'] + ' ' + df['abstract']
# df['categories'] = df['categories'].apply(lambda x: x.split(','))

In [12]:
df.iloc[0].categories

'cond-mat.str-el'

## Train dataset creation

In [13]:
df_2, df_unused = train_test_split(df, train_size=0.6)  # take only 60% of dataset
df_train, df_test = train_test_split(df_2, train_size=0.8)

df.size, df_train.size, df_test.size

(35000, 16800, 4200)

In [14]:
def get_tokenizer(tokenizer_model):
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    tokenizer = AutoTokenizer.from_pretrained(tokenizer_model)
    return tokenize_function, tokenizer

tokenize_function, tokenizer = get_tokenizer('bert-base-uncased')

In [15]:
mlb = MultiLabelBinarizer()
# mlb.fit([[(k,v) for k, v in CATEGORIES.items()]]) #df_train['categories'])
mlb.fit([list(CATEGORIES.keys())]) #df_train['categories'])
mlb.classes_[:10]

array(['astro-ph', 'astro-ph.CO', 'astro-ph.EP', 'astro-ph.GA',
       'astro-ph.HE', 'astro-ph.IM', 'astro-ph.SR', 'cond-mat.dis-nn',
       'cond-mat.mes-hall', 'cond-mat.mtrl-sci'], dtype=object)

In [16]:
def preprocess_data(examples):
    # take a batch of texts
    text = examples["text"]

    # encode them
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)

    encoded_categories = mlb.transform([c.split(',') for c in examples['categories']]).astype(float)

    encoding["labels"] = encoded_categories

    return encoding

In [17]:
df_train_hf = Dataset.from_pandas(df_train[['text', 'categories']])
tokenized_train = df_train_hf.map(preprocess_data, batched=True)

df_test_hf = Dataset.from_pandas(df_test[['text', 'categories']])
tokenized_test = df_test_hf.map(preprocess_data, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

/home/paulo/.local/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['econ.GN', 'econ.TH', 'eess.SY'] will be ignored
  warnings.warn(
/home/paulo/.local/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['eess.SY'] will be ignored
  warnings.warn(
/home/paulo/.local/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['econ.GN', 'eess.SY'] will be ignored
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?ba/s]

/home/paulo/.local/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['econ.TH', 'eess.SY'] will be ignored
  warnings.warn(


In [18]:
# Get inverse transform, as an example

print("Reversed", mlb.inverse_transform(np.asarray(tokenized_test[0]['labels']).reshape(1, -1)))
print("Original categories", tokenized_test[0]['categories'])

Reversed [('cond-mat.str-el',)]
Original categories cond-mat.str-el


In [19]:
# Store multilabel binarizer as a pickle file

!rm -r checkpoint
!mkdir checkpoint
with open('checkpoint/mlb.pkl', 'wb') as handle:
    pickle.dump(mlb, handle, protocol=pickle.HIGHEST_PROTOCOL)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Training multi label class model

In [20]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(mlb.classes_),
    problem_type="multi_label_classification"
)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [24]:
# Adaptation: https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb 
# 

# Define batch size according to your GPU RAM
batch_size = 8
nb_epochs = 1 # DEMONSTRATIVE PURPOSES
metric_name = "f1"

args = TrainingArguments(
    f"paper-multilabel-finetuning",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size//2,
    num_train_epochs=nb_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    eval_accumulation_steps=1,
)

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    return multi_label_metrics(
        predictions=p.predictions, 
        labels=p.label_ids)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [25]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [26]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, categories, __index_level_0__. If text, categories, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/paulo/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2400
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 100
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/home/paulo/.local/l

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.171790,0.000000,0.500000,0.000000


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, categories, __index_level_0__. If text, categories, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 600
  Batch size = 12
Saving model checkpoint to paper-multilabel-finetuning/checkpoint-100
Configuration saved in paper-multilabel-finetuning/checkpoint-100/config.json
Model weights saved in paper-multilabel-finetuning/checkpoint-100/pytorch_model.bin
tokenizer config file saved in paper-multilabel-finetuning/checkpoint-100/tokenizer_config.json
Special tokens file saved in paper-multilabel-finetuning/checkpoint-100/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from paper-multilabel-finetuning/checkpoint-100 (score: 0.0).


TrainOutput(global_step=100, training_loss=0.2128754997253418, metrics={'train_runtime': 35.1738, 'train_samples_per_second': 68.233, 'train_steps_per_second': 2.843, 'total_flos': 158080663756800.0, 'train_loss': 0.2128754997253418, 'epoch': 1.0})

In [27]:
eval = trainer.evaluate()
eval

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, categories, __index_level_0__. If text, categories, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 600
  Batch size = 12
/home/paulo/.local/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.1717897355556488,
 'eval_f1': 0.0,
 'eval_roc_auc': 0.5,
 'eval_accuracy': 0.0,
 'eval_runtime': 4.9813,
 'eval_samples_per_second': 120.45,
 'eval_steps_per_second': 10.038,
 'epoch': 1.0}

## Perform inference on a given text sample

In [28]:
text = df['text'].iloc[5]
categories = df['categories'].iloc[5]
print(categories)

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k, v in encoding.items()}

outputs = trainer.model(**encoding)
logits = outputs.logits

hep-ph,hep-ex


In [31]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.3)] = 1

In [32]:
print(text)
print(mlb.inverse_transform(predictions.reshape(1, -1)))

Using Integral Dispersion Relations to Extend the LHC Reach for New
  Physics   Many models of electroweak symmetry breaking predict new particles with
masses at or just beyond LHC energies. Even if these particles are too massive
to be produced on-shell at the LHC, it may be possible to see evidence of their
existence through the use of integral dispersion relations (IDRs). Making use
of Cauchy's integral formula and the analyticity of the scattering amplitude,
IDRs are sensitive in principle to changes in the cross section at arbitrarily
large energies. We investigate some models of new physics. We find that a
sudden, order one increase in the cross section above new particle mass
thresholds can be inferred well below the threshold energy. On the other hand,
for two more physical models of particle production, we show that the reach in
energy and the signal strength of the IDR technique is greatly reduced. The
peak sensitivity for the IDR technique is shown to occur when the new part

In [33]:
trainer.save_model(output_dir='./checkpoint')

Saving model checkpoint to ./checkpoint
Configuration saved in ./checkpoint/config.json
Model weights saved in ./checkpoint/pytorch_model.bin
tokenizer config file saved in ./checkpoint/tokenizer_config.json
Special tokens file saved in ./checkpoint/special_tokens_map.json


In [34]:
with open('./checkpoint/model_info.json', 'w') as f:
    f.write(json.dumps(eval, indent=4))